# How to import brush masks to labelstudio
## Author: Gabriel Carneiro (gabri14el)

In the first step you must run the label studio defining two environment variables:


```bash

LABEL_STUDIO_LOCAL_FILES_SERVING_ENABLED=true LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT=/home/gabriel/ label-studio

```


In this example, the label studio is going to recognize the __'/home/gabriel'__ as the base path to serve files into its own service. I did not test putting my dataset in several directories, i just put it in __'/home/gabriel/castanheiros'__.

Following, in the labelstudio go to your **project -> settings -> cloud storage** and click in **Add Source Storage**.  In Storage type select local files. In absolute local path put the path to your dataset and some name in Storage title (it is not useful). It is important that the option 'Treat every bucket object as a source file' to be untoggled. As showed in the example bellow (the example uses dataset1 instead castanheiros).

![](https://labelstud.io/images/local-storage-settings2.png)  

Then, click in **Add Storage**. Note that you **DO NOT MUST** click in Sync in the next page. 

To test if your local storage is ready, you can access the following link: ```http://localhost:8080/data/local-files/?d=\$YOUR_FIRST_FOLDER/\$SOME_FILE_THERE.jpg```. In my example, since my first folder after the absolute path is ```castanheiros```, inside it i have a file named ```DJI_0661-patch_47.jpg```, and my labl studion is running in the port 8080, i can retrieve the file using 


```
http://localhost:8080/data/local-files?d=castanheiros/DJI_0661-patch_47.jpg

```


This configuration is mandatory because the labstudio importing each image from some link. Others bucket as Amazon S3, Google Cloud and so on are supported. 


After, you must use the following code to convert your masks to the format that label studio can import. 

The code bellow to the job for us. Please, note that the library label_studio_converter must be installed. 

The following code must be placed in the labelling interface of your project:


```xml
<View>
      <Image name="image" value="$image" zoom="true"/>
      <BrushLabels name="tag" toName="image">
        <Label value="hedgehog" background="rgba(255, 0, 0, 0.7)"/>
      </BrushLabels>
</View>
```



In [1]:
import unittest
import urllib3
import json

from label_studio_converter.brush import encode_rle, image2annotation
import cv2
import os

In [2]:
dataset = r'/home/gabriel/Projects/Castanheiros/castanheiro-dataset'

#define the locations of images, masks, and where labels are going to be stored
images = os.path.join(dataset, 'images')
masks = os.path.join(dataset, 'masks')
masks_inverted = os.path.join(dataset, 'masks_inverted')
label_studio = os.path.join(dataset, 'labels')

os.makedirs(label_studio, exist_ok=True)
os.makedirs(masks_inverted, exist_ok=True)

#iterate over the images
for root, dirs, files in os.walk(images, topdown=False):
    for name in files:
        dic = {}
        file = os.path.join(root, name)
        img_name = name.split('.')[0]
        
        #this block invert the images, since IS MANDATORY that the background be represented by 0
        mask = cv2.imread(os.path.join(masks, img_name+'.png'))
        mask_inverted = cv2.bitwise_not(mask)
        cv2.imwrite(os.path.join(masks_inverted, img_name+'.png'), mask_inverted)
        
        #convert the annotation for the format accepeted by label studio
        #note that the class with 1 is named by 'hedgehog', keep the other values
        annotation = image2annotation(
            os.path.join(masks_inverted, img_name+'.png'),
            label_name='hedgehog',
            from_name='tag',
            to_name='image',
            model_version='v1',
            score=0.5,
        )
        
        #data must be a URL served by labelstudio, thus we are going to use the same url but without
        #defining host or port, only the subdirectories
        task = {
            'data': {'image': '/data/local-files/?d=castanheiros/'+name},
            'predictions': [annotation],
        }
        
        #save each json corresponding to a image
        json.dump(task, open(os.path.join(label_studio, img_name+'.json'), 'w'))
        


At the end of this step, for each image in your dataset you must have a correspondent json. This json contains the url opf the image and the annotation. You can import it in your project homepage! 

Example:


```json

{"data": {"image": "/data/local-files/?d=castanheiros/DJI_0661-patch_43.jpg"}, "predictions": [{"result": [{"id": "f1969bd3", "type": "brushlabels", "value": {"rle": [0, 22, 144, 0, 57, 27, 255, 255, 255, 0, 255, 255, 224, 31, 255, 252, 3, 255, 255, 128, 127, 255, 240, 15, 255, 254, 1, 255, 255, 192, 63, 255, 248, 7, 255, 255, 0, 255, 255, 224, 31, 255, 252, 3, 255, 255, 128, 127, 255, 240, 15, 255, 254, 1, 255, 255, 192, 63, 255, 248, 7, 255, 255, 0, 255, 255, 224, 31, 255, 252, 3, 255, 255, 128, 127, 255, 240, 15, 255, 254, 1, 227, 255, 192, 0], "format": "rle", "brushlabels": ["hedgehog"]}, "origin": "manual", "to_name": "image", "from_name": "tag", "image_rotation": 0, "original_width": 608, "original_height": 608}], "model_version": "v1", "score": 0.5}]}

```
